In [5]:
import cv2
import numpy as np
import random as rng
from utils import *
from crossword_utils import *

def euclidean_distance(point1, point2):
    # Calculates the euclidean distance between the point1 and point2
    # used to calculate the length of the four sides of the square 
    distance = np.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)
    return distance

def order_corner_points(corners):
    # The points obtained from contours may not be in order because of the skewness  of the image, or
    # because of the camera angle. This function returns a list of corners in the right order 
    sort_corners = [(corner[0][0], corner[0][1]) for corner in corners]
    sort_corners = [list(ele) for ele in sort_corners]
    x, y = [], []

    for i in range(len(sort_corners[:])):
        x.append(sort_corners[i][0])
        y.append(sort_corners[i][1])

    centroid = [sum(x) / len(x), sum(y) / len(y)]

    for _, item in enumerate(sort_corners):
        if item[0] < centroid[0]:
            if item[1] < centroid[1]:
                top_left = item
            else:
                bottom_left = item
        elif item[0] > centroid[0]:
            if item[1] < centroid[1]:
                top_right = item
            else:
                bottom_right = item

    ordered_corners = [top_left, top_right, bottom_right, bottom_left]

    return np.array(ordered_corners, dtype=np.float32)

def image_preprocessing(image, corners, h=500, w=500):
    # This function undertakes all the preprocessing of the image and return  
    ordered_corners = order_corner_points(corners)
    print("ordered corners: ", ordered_corners)

    top_left, top_right, bottom_right, bottom_left = ordered_corners

    # Determine the widths and heights  ( Top and bottom ) of the image and find the max of them for transform 

    width1 = euclidean_distance(bottom_right, bottom_left)
    width2 = euclidean_distance(top_right, top_left)

    height1 = euclidean_distance(top_right, bottom_right)
    height2 = euclidean_distance(top_left, bottom_right)

    width = max(int(width1), int(width2))
    height = max(int(height1), int(height2))

    # To find the matrix for warp perspective function we need dimensions and matrix parameters
    dimensions = np.array([[0, 0], [width, 0], [width, width],
                           [0, width]], dtype=np.float32)

    matrix = cv2.getPerspectiveTransform(ordered_corners, dimensions)

    # Return the transformed image
    transformed_image = cv2.warpPerspective(image, matrix, (width, width))

    # Now, chances are, you may want to return your image into a specific size. If not, you may ignore the following line
    transformed_image = cv2.resize(transformed_image, (w, h), interpolation=cv2.INTER_AREA)

    return transformed_image

def get_square_box_from_image(gray, h=500, w=500):
    # This function returns the top-down view of the puzzle in grayscale.

    blur = cv2.medianBlur(gray, 3)
    adaptive_threshold = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 11, 3)
    corners = cv2.findContours(adaptive_threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    corners = corners[0] if len(corners) == 2 else corners[1]
    corners = sorted(corners, key=cv2.contourArea, reverse=True)
    for corner in corners:
        length = cv2.arcLength(corner, True)
        approx = cv2.approxPolyDP(corner, 0.015 * length, True)
        print(approx)

        puzzle_image = image_preprocessing(image, approx, h, w)
        break

    return puzzle_image

# ####################################

from pathlib import Path
Path("temp").mkdir(parents=True, exist_ok=True)

image = cv2.imread('crossword-2.png')

# Transform source image to gray if it is not already
if len(image.shape) != 2:
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
else:
    gray = image

grayOriginal = gray.copy();

h, w = gray.shape
imgBlank = np.zeros((h, w, 3), np.uint8)

square = get_square_box_from_image(gray, h, w)

# Image Array for Display
imageArray = ([
    [image, grayOriginal, square], 
    [imgBlank, imgBlank, imgBlank]
] )
stackedImage = stackImages(imageArray, 0.25, 
[
    ['raw', 'gray', 'square'] , 
    ['blank', 'blank', 'blank']
])

show_wait_destroy("stacked", stackedImage)


[[[  78  344]]

 [[  14 2850]]

 [[2107 2884]]

 [[2151  401]]]
ordered corners:  [[  78.  344.]
 [2151.  401.]
 [2107. 2884.]
 [  14. 2850.]]


SystemExit: Pressed q - exiting ...